Notes
- Uses Metric Units
- Assumes P2 Values are available in GPI Export

TO DO
- Make code more accepting of both numeric and alphanumeric partnumbers
- Add class methods

##### Imports, File Setup

In [2]:
#Imports
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import shutil

In [3]:
# Dictionaries
curve_dict = {
    'Flow':'Q',
    'Head':'H',
    'Power':'P2',
    'Efficiency':'Eta2',
    'NPSH':'NPSH'
}

In [4]:
# This is the folder/file with the curve export csv
myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\1 - PROJECTS\! - HOLD\SP Integration\Curve PSD\Archive"
myFile = "PumpCurves.csv"
filePath = os.path.join(myDir, myFile)

##### Function Definitions and Class Definitions

In [5]:
# Affinity Laws function
def apply_affinity_laws(row, n2, n1):
    N1, N2 = n1, n2
    Q2 = row['Q'] * (N2 / N1)
    H2 = row['H'] * (N2 / N1)**2
    NPSH2 = row['NPSH'] * (N2 / N1)**2
    P22 = row['P2'] * (N2 / N1)**3
    # P12 = row['P1'] * (N2 / N1)**3
    rpm_pump_data = row['RPM(Pump data)']
    rpm_curve_nom = row['RPM(Curve nominal)']
    return pd.Series([Q2, H2, P22, NPSH2, rpm_pump_data, rpm_curve_nom], index=['Q','H','P2','NPSH', 'RPM(Pump data)','RPM(Curve nominal)'])

In [85]:
# Create class to hold each curve by product number/trim
class Curve:
    # def __init__(self, pumpModel: str, dataframe):
    def __init__(self, partNumber: str, dataframe):
        # self.name = pumpModel
        self.name = partNumber
        self.df = dataframe
        self.model = dataframe.iloc[0]['Product name']
        # self.pn = dataframe.iloc[0]['ProductNumber']
        self.frequency = dataframe.iloc[0]['Frequency']
        self.phase = dataframe.iloc[0]['Phase']
        self.trims = self.df['RPM(Curve nominal)'].unique().tolist()
        self.trim_curves = {} 


    def __repr__(self) -> str:
        return f"Pump Model(name={self.name}, dataframe=\n{self.df})"


    def return_xy_points(self, curveType:str):
        '''
        Returns 2 lists: 1 - flow values, 2 - Head or Power or NPSH
        
        Parameters:
        curveType (str): The list of y-values. Available options are:
            - 'Head'
            - 'Power'
            - 'NPSH'
        
        Raises:
        ValueError: If the action is not one of the available options.
        '''
        curve_types = ['Head','Power','NPSH']

        if curveType not in curve_types:
            raise ValueError(f"Invalid Curve Type. Available options are: {', '.join(curve_types)}")
        else:
            x_values = self.df['Q'].tolist()
            y_values = self.df[curve_dict[curveType]].tolist()
            return x_values, y_values
    

    def create_grouped_trim_curves(self):
        '''Group entire curve df by the trim/speed column'''
        grouped = self.df.groupby('RPM(Curve nominal)')
        for group_trim, trim_df in grouped:
            self.trim_curves[group_trim] = trim_df[['Q','H','P2','NPSH','RPM(Pump data)','RPM(Curve nominal)']]


    def create_new_trim_df(self, n2):
        """
        Takes in speed n2 and applies affinity laws to max available existing trim to calculate new curve data

        Output: Adds a dataframe to self.trim_curves dictionary

        """
        # Finds max existing trim and uses that as n1
        n1 = self.max_trim
        df1 = self.trim_curves[n1]

        # Apply the affinity laws to each row of df1 to create df2
        df2 = df1.apply(apply_affinity_laws, axis=1, args=(n2,n1))

        # Add new dataframe to dictionary trim_curves
        self.trim_curves.update({n2:df2}) 

        # Update Trims Property
        self.trims = list(self.trim_curves.keys())


    @property
    def max_trim(self):
        return max(self.trims)

##### Create objects from GPI Curve Export

In [ ]:
# This separates the GPI curve export by groups according to ProductNumber
df = pd.read_csv(filePath, sep=";", index_col=False, skip_blank_lines=False) # FOR SP
df = df.replace(',','.', regex=True)

# Drop NaN rows before grouping
df_non_nan = df.dropna(how='all')

# Forward fill productname and curve nominal columns for grouping 
df_non_nan['ProductNumber'] = df_non_nan['ProductNumber'].ffill()
df_non_nan['RPM(Curve nominal)'] = df_non_nan['RPM(Curve nominal)'].ffill()
df_non_nan['RPM(Pump data)'] = df_non_nan['RPM(Pump data)'].ffill()

# Group by the pump model column
grouped = df_non_nan.groupby('ProductNumber')

# Make a model from each group
group_objects = {}   # Structure: group_objects (dict) where each key = partnumber, value = Curve Object 

for partnumber, group_df in grouped:
    # print(f'pumpModel: {pumpModel}')
    # print(group_df)
    group_object = Curve(partNumber=partnumber, dataframe=group_df)
    group_object.create_grouped_trim_curves()
    group_objects[(f'{partnumber}')] = group_object
    

##### Creates list of matching curves (Currently only for single speed/trim) -> make this a class method


In [103]:
def find_duplicate_curveSets(dict_pn_curveDataframe):  
    """ Create list of PNs that share curve data """
    curveDataCopy = dict_pn_curveDataframe.copy()
    listOfLists = []
    
    for _, eachDF in dict_pn_curveDataframe.items():  
        listOfPNs = []
        for key, value in curveDataCopy.items():
            if value.equals(eachDF):
                listOfPNs.append(key)
        
        if listOfPNs not in listOfLists:
            listOfLists.append(listOfPNs)
    
    return listOfLists
    

In [105]:
pn_curve_dict = {}
models = []

for partnumber,CurveObj in group_objects.items():
    speeds = list(CurveObj.trim_curves.keys())[0]
    models.append(CurveObj.model)
    curve_df = CurveObj.trim_curves[speeds].reset_index(drop=True)
    pn_curve_dict.update({partnumber:curve_df})

matches_list = find_duplicate_curveSets(pn_curve_dict)


In [118]:
models_set = set(models)
# print(len(models_set))

550


In [116]:
for item in matches_list:
    if len(item) != 1:
        # Find model corresponding to each partnumber
        for eachPN in item:
            print(f'Model: {group_objects[eachPN].model}, Partnumber: {item}')
    

Model: 16S05-5, Partnumber: ['10014205', '10019205', '10154505', '10159505']
Model: 16S05-5, Partnumber: ['10014205', '10019205', '10154505', '10159505']
Model: 16S05-5, Partnumber: ['10014205', '10019205', '10154505', '10159505']
Model: 16S05-5, Partnumber: ['10014205', '10019205', '10154505', '10159505']
Model: 16S20-18, Partnumber: ['10014518', '10153018', '10153618']
Model: 16S20-18, Partnumber: ['10014518', '10153018', '10153618']
Model: 16S20-18, Partnumber: ['10014518', '10153018', '10153618']
Model: 16S30-24, Partnumber: ['10014524', '10153024', '10153624']
Model: 16S30-24, Partnumber: ['10014524', '10153024', '10153624']
Model: 16S30-24, Partnumber: ['10014524', '10153024', '10153624']
Model: 16S50-38, Partnumber: ['10114538', '10153038', '10153638']
Model: 16S50-38, Partnumber: ['10114538', '10153038', '10153638']
Model: 16S50-38, Partnumber: ['10114538', '10153038', '10153638']
Model: 16S15-14, Partnumber: ['10153014', '10153614', '10154514', '10159514']
Model: 16S15-14, Par

##### Creates new speed curve data using affinity laws -> Class method?

In [73]:
# # Iterate through list of models and and create new speed dataframes at n2
# n2 = 1800

# for object_name in group_objects:
#     model_object = group_objects[object_name]
#     model_object.create_new_trim_df(n2)

#### Write to PSD

In [74]:
# This points to the curve PSD template to be used
templateDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\3 - RESOURCES\32 GXS"
template = "SKB Blank Curve PSD - Power_Metric.xlsx"
template = os.path.join(templateDir, template)

# Create a local working copy to leave template unmodified
output_psd_file = 'SP - Curve PSD.xlsx'
workingCopy = shutil.copyfile(template, output_psd_file)
wb = load_workbook(workingCopy)      

##### Add 1 curve tab to workbook for every unique curve found (avoids duplicating data in SKB)


In [75]:
for object_name in group_objects:
    curve_pn = str(group_objects[object_name].pn)
    wb.copy_worksheet(wb['NEW']).title = curve_pn # Creates and renames blank PSD Tab as template for each new curve tab

##### Iterate through list of unique curve names, and populate the corresponding workbook tab


In [76]:
# Fill PSD 
for object_name in group_objects:
    curve_pn = str(group_objects[object_name].pn)
    curveSheet = wb[curve_pn]   
    # Fill Curve, power/eff, NPSH data
    baseCell = 'D7' 
    first_row_offset = 3

    # Fill Speed or trim in column A
    trims = group_objects[object_name].trims
    for index, eachSpeedTrim in enumerate(trims):
        cell_name = "{}{}".format('A', 10+index)
        curveSheet[cell_name].value = int(eachSpeedTrim)
        # print(f'For PN: {uniqueCurve[0]} -> cell: {cell_name} should have value: {int(eachSpeed)}. It currently has: {curveSheet[cell_name].value}')

        curve_data_df = group_objects[object_name].trim_curves[eachSpeedTrim].reset_index()
        for key, value in curve_data_df.iterrows():
            # print(f'first_row_offset {first_row_offset}, key: {key + 21*index}')
            curveSheet[baseCell].offset(first_row_offset + key, 0 + 21*index).value = value['Q']
            curveSheet[baseCell].offset(first_row_offset + key, 1 + 21*index).value = value['H']
            curveSheet[baseCell].offset(first_row_offset + key, 7 + 21*index).value = value['Q']
            # curveSheet[baseCell].offset(first_row_offset + key, 8 + 21*index).value = value['Eta1']
            curveSheet[baseCell].offset(first_row_offset + key, 8 + 21*index).value = value['P2']
            curveSheet[baseCell].offset(first_row_offset + key,14 + 21*index).value = value['Q']
            curveSheet[baseCell].offset(first_row_offset + key,15 + 21*index).value = value['NPSH']

##### Fill Curve Header Tab

In [77]:
newPumpFamilyName = "SP"
sheet = wb["Curve Header Data"]
sheet['B7'] = newPumpFamilyName
baseCell = 'A1' 
first_row_offset = 10

for index, object_name in enumerate(group_objects):
    df = group_objects[object_name].df
    max_speed = max(group_objects[object_name].trims)
    min_speed = min(group_objects[object_name].trims)
            
    sheet[baseCell].offset((first_row_offset + index), 0).value = str(group_objects[object_name].pn)
    sheet[baseCell].offset((first_row_offset + index), 2).value = df.iloc[0]['RPM(Pump data)']
    # sheet[baseCell].offset((first_row_offset + index), 3).value = group_objects[object_name].poles
    sheet[baseCell].offset((first_row_offset + index), 4).value = group_objects[object_name].frequency
    sheet[baseCell].offset((first_row_offset + index), 6).value = df.iloc[0]['RPM(Curve nominal)']
    sheet[baseCell].offset((first_row_offset + index), 7).value = min_speed
    sheet[baseCell].offset((first_row_offset + index), 8).value = max_speed
    sheet[baseCell].offset((first_row_offset + index), 21).value = min_speed

In [78]:
wb.save(workingCopy)